Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace
test-Jesse
westeurope
72f46e0e-1451-4b79-92cd-fc8f7797bda7


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-bike-experiment-1'
project_folder = './pipeline-bike-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-bike-experiment-1,Workspace,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [18]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-demo-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [16]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bikesharing Dataset"
description_text = "Bike Sharing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


# df = dataset.to_pandas_dataframe()
# df.describe()

### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [17]:
dataset.take(5).to_pandas_dataframe()

,instant,date,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,6,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,2,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,3,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [41]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'normalized_root_mean_squared_error',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "forecasting",
                             training_data=dataset,
                             time_column_name="date", 
                             label_column_name="cnt",  
                             path = project_folder,
                             enable_early_stopping= True,
                             #featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [42]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [43]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [44]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [45]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [094a92ce][bb6afafb-cfa4-48a1-a722-0109a1887e1f], (This step is eligible to reuse a previous run's output)
Submitted PipelineRun a8f92415-39d5-4b04-add3-4015126417e8
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-bike-experiment-1/runs/a8f92415-39d5-4b04-add3-4015126417e8?wsid=/subscriptions/72f46e0e-1451-4b79-92cd-fc8f7797bda7/resourcegroups/test-Jesse/workspaces/Workspace


In [46]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [47]:
pipeline_run.wait_for_completion()

PipelineRunId: a8f92415-39d5-4b04-add3-4015126417e8
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-bike-experiment-1/runs/a8f92415-39d5-4b04-add3-4015126417e8?wsid=/subscriptions/72f46e0e-1451-4b79-92cd-fc8f7797bda7/resourcegroups/test-Jesse/workspaces/Workspace
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 7b9b2680-2706-4574-a21e-546d2e700aa3
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-bike-experiment-1/runs/7b9b2680-2706-4574-a21e-546d2e700aa3?wsid=/subscriptions/72f46e0e-1451-4b79-92cd-fc8f7797bda7/resourcegroups/test-Jesse/workspaces/Workspace
StepRun( automl_module ) Status: NotStarted

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '7b9b2680-2706-4574-a21e-546d2e700aa3', 'target': 'cluster-demo-1', 'status': 'Completed', 'startTimeUtc': '2021-01-12T19:40:37.406248Z', 'endTimeUtc': '2021-01-12T19:40:40.67657Z', 'properties': {'azureml.reusedrunid': '41

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [20]:
metrics_output_name = 'metrics_output'
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Path already exists. Skipping download for .\azureml/41e0f09b-42a2-45a9-86f2-f359a07fb549/metrics_data


In [21]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,41e0f09b-42a2-45a9-86f2-f359a07fb549_15,41e0f09b-42a2-45a9-86f2-f359a07fb549_28,41e0f09b-42a2-45a9-86f2-f359a07fb549_3,41e0f09b-42a2-45a9-86f2-f359a07fb549_34,41e0f09b-42a2-45a9-86f2-f359a07fb549_20,41e0f09b-42a2-45a9-86f2-f359a07fb549_1,41e0f09b-42a2-45a9-86f2-f359a07fb549_33,41e0f09b-42a2-45a9-86f2-f359a07fb549_32,41e0f09b-42a2-45a9-86f2-f359a07fb549_26,41e0f09b-42a2-45a9-86f2-f359a07fb549_0,...,41e0f09b-42a2-45a9-86f2-f359a07fb549_5,41e0f09b-42a2-45a9-86f2-f359a07fb549_8,41e0f09b-42a2-45a9-86f2-f359a07fb549_30,41e0f09b-42a2-45a9-86f2-f359a07fb549_24,41e0f09b-42a2-45a9-86f2-f359a07fb549_23,41e0f09b-42a2-45a9-86f2-f359a07fb549_40,41e0f09b-42a2-45a9-86f2-f359a07fb549_17,41e0f09b-42a2-45a9-86f2-f359a07fb549_25,41e0f09b-42a2-45a9-86f2-f359a07fb549_18,41e0f09b-42a2-45a9-86f2-f359a07fb549_19
mean_absolute_error,[194.13021724818958],[146.46058823529415],[953.6],[1.9671641890006868],[232.80368663594473],[0.31174049520759584],[1.1743562403211398],[1.2430668916432297],[184.4045168067227],[691.3104671462055],...,[660.8857142857144],[0.003691553795033542],[0.9859977274554694],[0.0013394202468134607],[133.2155555555555],[0.0013394202468134607],[170.62973304473303],[193.49201680672272],[2.561325610078984],[210.24786096256685]
spearman_correlation,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],...,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0]
normalized_median_absolute_error,[0.02233435541281518],[0.01685004466581847],[0.1097100782328578],[0.00022631893568806797],[0.026783673105838096],[3.5865220341416916e-05],[0.00013510771287633916],[0.00014301275789728826],[0.021215429913336713],[0.07953410804719346],...,[0.07603379133521793],[4.247070633954834e-07],[0.00011343738235796933],[1.5409804956436503e-07],[0.015326225903768465],[1.5409804956436503e-07],[0.01963066417909952],[0.0222609315240132],[0.00029467620916693324],[0.024188663249259876]
normalized_root_mean_squared_log_error,[0.01366466277992622],[0.011089234646635755],[0.07355255485668191],[0.0001904009936876344],[0.017050914932548174],[2.0406612067675017e-05],[8.652299659201229e-05],[8.10058023945132e-05],[0.015027794416149116],[0.05252703117017291],...,[0.05413379748284958],[3.316625456338143e-07],[8.850305639934937e-05],[1.2040924163875155e-07],[0.011206611086424755],[1.2040924163875155e-07],[0.014146533929136474],[0.016010084739692388],[0.00016614513420297488],[0.01737775082236407]
r2_score,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
root_mean_squared_log_error,[0.08113129445075362],[0.06584018763136737],[0.43670407984373905],[0.0011304691035098457],[0.10123651218671252],[0.00012116031541129502],[0.0005137135710059582],[0.00048095629670008577],[0.08922462510476734],[0.3118690963052234],...,[0.3214089607663372],[1.96918226826881e-06],[0.0005254697934496732],[7.149066021838734e-07],[0.06653708755868148],[7.149066021838734e-07],[0.08399231127374698],[0.0950567840653667],[0.0009864546254579666],[0.10317703711885215]
root_mean_squared_error,[194.13021724818958],[146.46058823529415],[953.6],[1.9671641890006868],[232.80368663594473],[0.31174049520759584],[1.1743562403211398],[1.2430668916432297],[184.4045168067227],[691.3104671462055],...,[660.8857142857144],[0.003691553795033542],[0.9859977274554694],[0.0013394202468134607],[133.2155555555555],[0.0013394202468134607],[170.62973304473303],[193.49201680672272],[2.561325610078984],[210.24786096256685]
mean_absolute_percentage_error,[8.024240915032127],[6.578864198030724],[41.90871839259893],[0.11325417526538972],[10.177552128158805],[0.01212326867241115],[0.05138082751039945],[0.04813563179805289],[9.153887811458096],[40.80171670014461],...,[26.212179210428694],[0.00019703208374838554],[0.05259537801513261],[7.153195309490105e-05],[6.837712999486412],[7.153195309490105e-05],[8.729712077960773],[9.831553732412608],[0.09885873871359721],[10.764507193205713]
explained_variance,[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[

### Retrieve the Best Model

In [23]:
# Retrieve best model from Pipeline Run
best_model_output_name = 'best_model_output'
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Path already exists. Skipping download for .\azureml/41e0f09b-42a2-45a9-86f2-f359a07fb549/model_data


In [24]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

AttributeError: Can't get attribute 'FrozenNDArray' on <module 'pandas.core.indexes.frozen' from 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\pandas\\core\\indexes\\frozen.py'>

In [ ]:
best_model.steps

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [25]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Workspace
test-Jesse
westeurope
72f46e0e-1451-4b79-92cd-fc8f7797bda7


In [35]:
# NOTE: update these to match your existing experiment name and a previous experiment
experiment_name = 'Hyperparam-search'
project_folder = './pipeline-bike-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
Hyperparam-search,Workspace,Link to Azure Machine Learning studio,Link to Documentation


In [36]:
from azureml.pipeline.core import PipelineRun

run_id = "b05118c5-ab36-418c-a774-ee0058d772ad"
pipeline_run = PipelineRun(experiment, run_id)

In [37]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bikesharing Train-3", description="Training bikesharing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bikesharing Train-3,0442ad27-6ce2-40dc-be07-1fca5d050b11,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [9]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [29]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bike-rest-endpoint"}
                        )

In [30]:
rest_endpoint

'https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/72f46e0e-1451-4b79-92cd-fc8f7797bda7/resourceGroups/test-Jesse/providers/Microsoft.MachineLearningServices/workspaces/Workspace/PipelineRuns/PipelineSubmit/594ed5a2-823c-4162-9b5a-6fa52c4da58d'

In [31]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  9808de2a-e3c4-48ce-ac33-a46d689bfc0e


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [32]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bike-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …